# Skin Cancer Classification

En el siguiente notebook se expone el desarrollo de una CNN para clasificar fotografías de lunares en función de si son cancerígenas o no. El dataset está compuesto por fotografías en formato ".jpg" separadas en carpetas según si pertenecen al set de entrenamiento o de test y según si son benignas o malignas. El dataset parece complicado puesto que las fotografías han sido tomadas desde distintos ángulos, con distinta luz y algunas imágenes se ven peor debido a que el paciente tenia vello en la zona del lunar.

El siguiente código refleja el mejor modelo encontrado para la tarea, como este no muestra un rendimiento suficientemente bueno no ha sido incluido en el trabajo.

### Imports

In [15]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras import regularizers 
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Dropout

np.random.seed(5)

### Read data function

In [16]:
def read(folder_path):
    data_path = os.path.join(folder_path,'*jpg')
    folder = glob.glob(data_path)
    matrix = []
    for f in folder:
        img = np.asarray(Image.open(f).convert("RGB"))
        matrix.append(img)
    matrix = np.asarray(matrix)
    return matrix

### Model function

2 capas de 3 filtros de tamaño 7 y, padding=same y strides=2. Maxpoolin tras las coapas convolucionales y luego una arquitectura de 6 capas fully connected.

In [17]:
def build_model():
    #Model
    model = Sequential()

    model.add(Conv2D(3, kernel_size=7,padding='same', strides=2, activation='relu', input_shape=(224,224,3)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))#model.add(BatchNormalization())#model.add(Dropout(0.25))

    model.add(Conv2D(3, kernel_size=3,padding='same', strides=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same', data_format=None))#model.add(BatchNormalization())#model.add(Dropout(0.25))

    #Normal layers
    model.add(Flatten())#model.add(Dropout(0.25))
    model.add(Dense(450, activation='relu',kernel_regularizer=regularizers.l2(0.01)))# model.add(Dropout(0.25))
    model.add(Dense(200, activation='relu',kernel_regularizer=regularizers.l2(0.01)))#model.add(Dropout(0.25))
    model.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01)))#model.add(Dropout(0.25))
    model.add(Dense(30, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(2, activation='softmax')) 
    return model

### Load data

In [18]:
benign_train_folder = 'data/train/benign'
malignant_train_folder = 'data/train/malignant'

benign_test_folder = 'data/test/benign'
malignant_test_folder = 'data/test/malignant'

### Split data

In [19]:
X_benign_train = read(benign_train_folder)
X_malignant_train = read(malignant_train_folder)
X_benign_test = read(benign_test_folder)
X_malignant_test = read(malignant_test_folder)

### Create labels

In [20]:
Y_benign_train = np.zeros(X_benign_train.shape[0])
Y_malignant_train = np.ones(X_malignant_train.shape[0])
Y_benign_test = np.zeros(X_benign_test.shape[0])
Y_malignant_test = np.ones(X_malignant_test.shape[0])

### Merge and Shuffle data

In [21]:
X_train = np.concatenate((X_benign_train, X_malignant_train), axis = 0)
Y_train = np.concatenate((Y_benign_train, Y_malignant_train), axis = 0)
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

X_test = np.concatenate((X_benign_test, X_malignant_test), axis = 0)
Y_test = np.concatenate((Y_benign_test, Y_malignant_test), axis = 0)
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

### Turn labels into one hot encoding.


In [22]:
Y_train = to_categorical(Y_train, num_classes= 2)
Y_test = to_categorical(Y_test, num_classes= 2)

#### Normalization

In [23]:
X_train = X_train/255.
X_test = X_test/255.

### CNN

In [24]:
model = build_model()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train,Y_train,validation_data=(X_test, Y_test),batch_size=10,epochs=50)

Train on 2637 samples, validate on 660 samples
Epoch 1/50
2637/2637 [==============================] - 9s 3ms/step - loss: 1.9190 - accuracy: 0.5446 - val_loss: 0.7288 - val_accuracy: 0.5455
Epoch 2/50
2637/2637 [==============================] - 9s 3ms/step - loss: 0.6764 - accuracy: 0.6166 - val_loss: 0.6064 - val_accuracy: 0.6939
Epoch 3/50
2637/2637 [==============================] - 8s 3ms/step - loss: 0.5776 - accuracy: 0.7399 - val_loss: 0.5463 - val_accuracy: 0.7712
Epoch 4/50
2637/2637 [==============================] - 8s 3ms/step - loss: 0.5333 - accuracy: 0.7649 - val_loss: 0.4958 - val_accuracy: 0.7606
Epoch 5/50
2637/2637 [==============================] - 8s 3ms/step - loss: 0.5057 - accuracy: 0.7793 - val_loss: 0.5123 - val_accuracy: 0.7470
Epoch 6/50
2637/2637 [==============================] - 8s 3ms/step - loss: 0.4985 - accuracy: 0.7876 - val_loss: 0.5346 - val_accuracy: 0.7985
Epoch 7/50
2637/2637 [==============================] - 8s 3ms/step - loss: 0.4789 - accu

El resultado final es de una 90% en el entrenamiento y 82% en el test.

In [25]:
test_loss, test_acc = model.evaluate(X_test, Y_test,batch_size=10)
print('Test Acc: ',test_acc)


660/660 [==============================] - 1s 2ms/step
Test Acc:  0.8227272629737854
